# Run Individual Galaxies with OMEGA+ (the code that runs on each GAMMA branch)

In [ ]:
# Import python modules
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import copy
import imp
import os

In [ ]:
# Define path to the codes
sygmadir = '/Users/benoitcote/Desktop/Carleen/Code_Carleen/NuPyCEE'
jinapydir = '/Users/benoitcote/Desktop/Carleen/Code_Carleen/JINAPyCEE'

# Set the environments to let Python knows where are the codes
os.environ['SYGMADIR'] = sygmadir
os.environ['JINAPYDIR'] = jinapydir

# Import the codes
sygma = imp.load_source('sygma', sygmadir+'/sygma.py')
omega = imp.load_source('omega', sygmadir+'/omega.py')
gamma = imp.load_source('gamma', jinapydir+'/gamma.py')

omega_plus = imp.load_source('omega_plus', jinapydir+'/omega_plus.py')

## Set Parameters

In [ ]:
# Function to return the correctly-scaled parameters
# for trees, as a function of their final dark matter mass
def get_input_sfe(m_DM_final):
    return sfe / (m_DM_final / m_DM_0_ref)**sfe_m_index
def get_input_mass_loading(m_DM_final):
    c_eta = mass_loading * m_DM_0_ref**(exp_ml/3.0)
    return c_eta * m_DM_final**(-exp_ml/3.0)

In [ ]:
# SHOULD NOT be modified
m_DM_0_ref = 1.0e12
C17_eta_z_dep = False
DM_outflow_C17 = True
sfe_m_dep = True
t_star = -1
t_inflow = -1

# Parameters to explore
t_ff_index = 4.36668122e-01 
f_t_ff = 7.53074817e+00 
sfe_m_index = 0
sfe = 9.47334087e-02
f_dyn = 5.98605814e-02
t_sf_z_dep = 0.5
exp_ml = 1.95785862e+00 
mass_loading = 0.2
f_halo_to_gal_out =  8.06626165e-01
nb_1a_per_m = 1.26060077e-03

# Extra parameter when OMEGA+ is not connected to GAMMA
# Mass of the dark matter halo hosting the central galaxy
m_DM_0 = 1.0e12  # [Msun]

# Define the dictionary containing GAMMA parameters
# Here I replaced host_tree.m_DM_0 by m_CM_0
kwargs = {"print_off":False, "t_ff_index":t_ff_index, "f_t_ff":f_t_ff,\
          "sfe_m_index":sfe_m_index, "sfe":get_input_sfe(m_DM_0), \
          "f_dyn":f_dyn, "t_sf_z_dep":t_sf_z_dep, "exp_ml":exp_ml, \
          "mass_loading":get_input_mass_loading(m_DM_0),\
          "f_halo_to_gal_out":f_halo_to_gal_out, "nb_1a_per_m":nb_1a_per_m,\
          "C17_eta_z_dep":C17_eta_z_dep, "DM_outflow_C17":DM_outflow_C17,\
          "sfe_m_dep":sfe_m_dep, "t_star":t_star, "t_inflow":t_inflow,\
          "m_DM_0":m_DM_0}

## Run OMEGA+

In [ ]:
# special_timesteps is the number of timesteps (they get longer as time increase)
# You can reduce the number of timesteps if you want to gain time ..
op = omega_plus.omega_plus(special_timesteps=250, **kwargs)

## Plot Outputs

### Mass of gas and stars as a function of time

In [ ]:
%matplotlib nbagg

# Extract the time array
t = np.array(op.inner.history.age) / 1.0e9
nb_t = len(t)

# Extract the mass of stars inside the galaxy
m_star = np.zeros(nb_t-1)
m_star[0] = op.inner.history.m_locked[0] - sum(op.inner.mdot[0])
for i_t in range(1,nb_t-1):
    m_star[i_t] = m_star[i_t-1] + op.inner.history.m_locked[i_t] - sum(op.inner.mdot[i_t])

# Extract the mass of gas inside the galaxy
m_gas_in = np.zeros(nb_t)
for i_t in range(nb_t):
    m_gas_in[i_t] = sum(op.inner.ymgal[i_t])
    
# Extract the mass of gas in the circumgalactic medium (gas surrounding the galaxy)
m_gas_cgm = np.zeros(nb_t)
for i_t in range(nb_t):
    m_gas_cgm[i_t] = sum(op.ymgal_outer[i_t])
    
# Extract the mass of gas lost forever (lost from the circumgalactic medium)
m_gas_lost = np.zeros(nb_t-1)
m_gas_lost[0] = op.m_lost_t[0]
for i_t in range(1,nb_t-1):
    m_gas_lost[i_t] = m_gas_lost[i_t-1] + op.m_lost_t[i_t]

# Plot quantities
plt.plot(t[:-1], m_star, label='Stars in galaxy')
plt.plot(t,      m_gas_in, label='Gas in galaxy')
plt.plot(t,      m_gas_cgm, label='Gas in CGM')
plt.plot(t[:-1], m_gas_lost, label='Gas lost')

# Adjust the y axis limits
plt.ylim(1e5, 1e12)

# Labels, legend, and scaling
plt.title('Dark matter halo mass = '+str('%.2E'%m_DM_0)+' Msun', fontsize=12)
plt.xlabel('Time [Gyr]')
plt.ylabel('Mass [M$_\odot$]')
plt.yscale('log')
plt.legend(fontsize=12, loc='best')

### Star formation rate as a function of time

In [ ]:
%matplotlib nbagg
op.inner.plot_star_formation_rate(color='k', label='Test 1')
#plt.yscale('log')

### [Fe/H] as a function of time

In [ ]:
%matplotlib nbagg
op.inner.plot_spectro(color='k', label='Test 1')
plt.ylim(-5,1)
plt.xlim(1e7,13e9)
plt.xscale('log')

### Plot star formation efficiency

In [ ]:
%matplotlib nbagg

# Extract the star formation efficiency [yr^-1]
sfe_t = np.zeros(nb_t-1)
for i_t in range(nb_t-1):
    sfe_t[i_t] = op.sfe[i_t] / op.inner.t_SF_t[i_t]

# Plot quantity
plt.plot(t[:-1], sfe_t)

# Labels
plt.xlabel('Time [Gyr]')
plt.ylabel('Star formation efficiency [yr$^{-1}$]')